In [2]:
!ls ../core

__pycache__          backtest_engine.py   portfolio_manager.py


## 1. Backtest Engine

### 1.1 Original `BacktestEngine` Object

In [ ]:
class BacktestEngine:
    
    def __init__(self, data, strategy, initial_cash = 1000):
        """
        Initialize Backtest Engine Object
        
        @param data: a data frame with columns of "Date" and "Close", it's genrated by csv_loader in data_loader directory
        @param strategy: a strategy object (e.g. RSIStrategy object)
        @param initial_cash: default 1000
        """
        
        self.data = data
        self.strategy = strategy
        self.initial_cash = initial_cash
        
        
        self.cash_stack = [initial_cash * 0.2] * 5
        self.position_stack = []
        self.portfolio_value_list = [] #Track portfolio value over time
        
        
    def run(self):
        """
        Generate trading signals and Execute trades based on signals.
        Protfolio value will be recorded over time.
        """
        
        # Generate the trading signals for each close price
        signal_list = self.strategy.generate_signals(self.data)
        
        for i in range(len(self.data)):
            signal = signal_list[i]
            price = self.data["Close"].iloc[i]
            
            if signal == 1 and len(self.cash_stack) > 0: # buy signal
                cash = self.cash_stack.pop()
                shares_to_buy = cash / price
                
                self.position_stack.append(shares_to_buy)
                
            elif signal == -1 and len(self.position_stack) > 0: #sell signal
                shares_to_sell = self.position_stack.pop()
                cash = price * shares_to_sell
                
                self.cash_stack.append(cash)
                
            # calculate current portfolio value   
            current_portfolio_value = sum(self.cash_stack) + sum(self.position_stack) * price
            self.portfolio_value_list.append(current_portfolio_value)
            
            
    def get_results(self):
        """
        retrieve the portfolio values over the backtesting period
        
        @return: A list of portfolio values at each time step
        """
        return self.portfolio_value_list
        
        
    
    def calculate_performance(self):
        """
        calculate the total return

        @return: a dictionary that contains total return information
        """
        
        final_value = self.portfolio_value_list[-1]
        
        total_return = (final_value - self.initial_cash) / self.initial_cash
        
        return {"Total Return": total_return}

### 1.2 `BacktestEngine` Object after introducing `PortfolioManager` Object in `portfolio_manager.py`

In [ ]:
from portfolio_manager import PortfolioManager


class BacktestEngine:
    
    def __init__(self, data, strategy, initial_cash = 1000):
        """
        Initialize Backtest Engine Object
        
        @param data: a data frame with columns of "Date" and "Close", it's genrated by csv_loader in data_loader directory
        @param strategy: a strategy object (e.g. RSIStrategy object)
        @param initial_cash: default 1000
        """
        
        self.data = data
        self.strategy = strategy
        self.initial_cash = initial_cash
        
        self.portfolio_manager = PortfolioManager(initial_cash)

        self.portfolio_value_list = [] #Track portfolio value over time
        
        
    def run(self):
        """
        Generate trading signals and Execute trades based on signals.
        Protfolio value will be recorded over time.
        """
        
        # Generate the trading signals for each close price
        signal_list = self.strategy.generate_signals(self.data)
        
        for i in range(len(self.data)):
            signal = signal_list[i]
            price = self.data["Close"].iloc[i]
            
            if signal == 1: # buy signal
                success = self.portfolio_manager.buy(price)
                if not success:
                    print(f"Buy failed at {self.data.index[i]}: Insufficient cash")
                
            elif signal == -1: #sell signal
                success = self.portfolio_manager.sell(price)
                if not success:
                    print(f"Sell failed at {self.data.index[i]}: Insufficient position")
   
            # calculate current portfolio value   
            current_portfolio_value = self.portfolio_manager.get_portfolio_value(price)
            self.portfolio_value_list.append(current_portfolio_value)
            
            
    def get_results(self):
        """
        retrieve the portfolio values over the backtesting period
        
        @return: A list of portfolio values at each time step
        """
        return self.portfolio_value_list
        
        
    
    def calculate_performance(self):
        """
        calculate the total return

        @return: a dictionary that contains total return information
        """
        
        final_value = self.portfolio_value_list[-1]
        
        total_return = (final_value - self.initial_cash) / self.initial_cash
        
        return {"Total Return": total_return}

## 2. Portfolio Manager

### `PorfolioManager` Object in `portfolio_manager.py`

1. `__init__`:
   - initialize cash stack with given initial cash
   - initialize empty position stack
2. `buy()`:
3. `sell()`:
4. `get_portfolio_value()`

In [ ]:
class PortfolioManager:

    def __init__(self, initial_cash, transaction_fee = 0.001, n_splits = 5):

        self.cash_stack = [initial_cash / n_splits] * n_splits
        self.position_stack = []
        self.transaction_fee = 0.001


    def buy(self, current_price):

        if len(self.cash_stack) > 0:
            cash = self.cash_stack.pop()
            shares_to_buy = cash / (current_price * (1 + self.transaction_fee))
            
            self.position_stack.append(shares_to_buy)
            return True
        return False


    def sell(self, current_price):

        if len(self.position_stack) > 0:
            shares_to_sell = self.position_stack.pop()
            cash = shares_to_sell * current_price * (1 - self.transaction_fee)

            self.cash_stack.append(cash)
            return True
        return False


    def get_portfolio_value(self, current_price):

        return sum(self.cash_stack) + sum(self.position_stack) * current_price
        